In [ ]:
import numpy as np

# pydrostat
Muscular hydrostat simulation using constrained dynamics

This notebook demonstrates a simulation of a muscular hydrostat arm. 



# 1) Define a structure to simulate

The structure is the arm or tentacle or cell being simulated. It is any set of vertices connected by edges, faces, and cells (although it's not required to have any of those connections). This notebook will demonstrate with an arm made of cubic cells. 

In [ ]:
from pydrostat.structures import arm_builder

arm_height = 6
arm_builder_1 = arm_builder.CubicArmBuilder(arm_height)
arm_builder_2 = arm_builder.CubicArmBuilder(arm_height)


# 2) Define an environment for the structure

An environment has food and obstacles. The food emmits a scent which diffuses through the environment around the obstacles (ie obstacles block the scent). This requires a simulation of the diffusion of the scent which approaches some steady state distribution of scent concentration.

Add this environment to the arm builder.

In [ ]:
from pydrostat.environment import environment
from pydrostat.environment.obstacles import convex_polytope
from pydrostat.environment import food

food_loc = np.array([[-3, 1, 3]])
# food_loc = np.array([3.5, 0, 0])
food_mag = 1
food_instance = food.Food(food_loc, food_mag)

rect_obst = convex_polytope.build_rectangular_obstacle([-2.25, -1, -1], [-1.5 ,9, 9])
# rect_obst = convex_polytope.build_rectangular_obstacle([-2.25, -9, -1], [-1.5 ,1, 9])

env = environment.Environment(dim = 3, obstacles = [rect_obst], foods = [food_instance])
arm_builder_1.set_environment(env)

arm_builder_2.set_environment(env)


# 3) Define sensors and a controller for the structure

A structure takes sensory measurements of its environment and passes the sense data to a controller. Each type of controller may require a specific type of sensor data, or some combination of sensor data. 

In [ ]:
from pydrostat.sensing.vertex_chemoceptors import VertexChemoceptors
from pydrostat.control.hand_tuned_gradient import HandTunedGradient
from pydrostat.control.hand_tuned_gradient import HandTunedGradient2

arm_builder_1.add_sensor(VertexChemoceptors())
arm_builder_1.set_controller(HandTunedGradient())

arm_builder_2.add_sensor(VertexChemoceptors())
arm_builder_2.set_controller(HandTunedGradient2())


# 4) Add constraints

Each constraint defines some sort of equality or inequality constraint that must be maintained throughout the simulation. This can be something like constant volume of a cell, minimum or maximum edge length, etc. 

In [ ]:
from pydrostat.constraint.constant_volume import ConstantVolumeCommon
from pydrostat.constraint.fixed_vertex import FixedVertex
from pydrostat.constraint.planar_faces import PlanarFacesCommon
from pydrostat.constraint.edge_length import ClipLength

arm_builder_1.add_constraint(ConstantVolumeCommon())
arm_builder_1.add_constraint(FixedVertex([0,1,2,3]))
arm_builder_1.add_constraint(PlanarFacesCommon())
arm_builder_1.add_constraint(ClipLength(0.5, 2.5))

arm_builder_2.add_constraint(ConstantVolumeCommon())
arm_builder_2.add_constraint(FixedVertex([0,1,2,3]))
arm_builder_2.add_constraint(PlanarFacesCommon())
arm_builder_2.add_constraint(ClipLength(0.5, 2.5))

# 5) Build the arm

In [ ]:
arm_1 = arm_builder_1.construct_arm()
arm_2 = arm_builder_2.construct_arm()

In [ ]:
# arm.apply_external_forces(4, np.array([-1,-1,0])*2)
# arm.apply_external_forces(4, np.array([0,0,-1])*0.5)

In [ ]:
from pydrostat.render.display import DisplayStructure

render_display = DisplayStructure(env, [arm_1, arm_2], dt=1/60)
render_display.main_loop(simulating=False)